In [1]:
import sys
import os
from logging import getLogger, ERROR

notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Fix module imports
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Disable Hugging Face warnings
getLogger("transformers.modeling_utils").setLevel(ERROR)

In [30]:
from pandas import read_csv, Series
from datasets import Dataset
from sklearn.metrics import f1_score
import numpy
from transformers import GPT2Tokenizer
from concrete.ml.sklearn import SGDClassifier
from concrete.fhe.compilation import Configuration
from model.qgpt2_models import SingleHeadQGPT2Model
import time

tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
model = SingleHeadQGPT2Model.from_pretrained("openai-community/gpt2", n_bits=8,use_cache=False)

for param in model.parameters():
    param.requires_grad = False

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [4]:
df = read_csv("../data/Tweets.csv")

df["airline_sentiment"] = df["airline_sentiment"].replace(
    ["negative", "neutral", "positive"], [0, 1, 2]
)

dataset = Dataset.from_pandas(df)
dataset = dataset.select_columns(["text", "airline_sentiment"])
dataset = dataset.rename_column("airline_sentiment", "label")

ds_dict = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = ds_dict["train"]
eval_ds = ds_dict["test"]

In [25]:
# Function that transforms a list of texts to their representation
# learned by the transformer.

def get_hidden_states(
    inputs: list,
    transformer_model,
    tokenizer: GPT2Tokenizer,
    device: str = "cuda",
):
    # Tokenize each text in the list one by one
    tokenized = map(lambda x: tokenizer.encode(x, return_tensors="pt"), inputs)

    # Send the model to the device
    transformer_model = transformer_model.to(device)
    output_hidden_states_list = []

    for tokenized_x in tokenized:
        # Pass the tokens through the transformer model and get the hidden states
        # Only keep the last hidden layer state for now
        output_hidden_states = transformer_model(tokenized_x.to(device), output_hidden_states=True)[0]
        # Average over the tokens axis to get a representation at the text level.
        output_hidden_states = output_hidden_states.mean(dim=1)
        output_hidden_states = output_hidden_states.detach().cpu().numpy()
        output_hidden_states_list.append(output_hidden_states)

    return numpy.concatenate(output_hidden_states_list, axis=0)

hidden_states = get_hidden_states(train_ds["text"], model, tokenizer)
x_test_states = get_hidden_states(eval_ds["text"], model, tokenizer)

In [6]:
numpy.savetxt("train_hidden_states.csv", hidden_states, delimiter=",")
numpy.savetxt("test_hidden_states.csv", x_test_states, delimiter=",")

## Using encrypted SGDClassifier

In [32]:
classifier = SGDClassifier(
            random_state=42,
            max_iter=1000,
            fit_encrypted=True,
            parameters_range=(-2**8, 2**8),
        )

In [36]:
y_train = Series(train_ds["label"])
y_test = Series(eval_ds["label"])

classifier.fit(hidden_states, y_train)

classifier.compile(hidden_states, configuration=Configuration(show_statistics=True))

start = time.perf_counter()
y_pred = classifier.predict(x_test_states, fhe="execute")
end = time.perf_counter()


f1 = f1_score(
    y_test, y_pred, average="macro"
)

f1s = f1_score(
    y_test, y_pred, average=None
)

print(f"Run time: {end - start:.4f} seconds")
print(f"Macro F1: {f1:.4f}")
print(f"F1 score for negative class: " f"{f1s[0]:.4f}")
print(f"F1 score for neutral class: " f"{f1s[1]:.4f}")
print(f"F1 score for positive class: " f"{f1s[2]:.4f}")



Statistics
--------------------------------------------------------------------------------
size_of_inputs: 8792064
size_of_outputs: 34344
programmable_bootstrap_count: 0
key_switch_count: 0
packing_key_switch_count: 0
clear_addition_count: 3
clear_addition_count_per_parameter: {
    LweSecretKeyParam(dimension=1430): 3
}
encrypted_addition_count: 3075
encrypted_addition_count_per_parameter: {
    LweSecretKeyParam(dimension=1430): 3075
}
clear_multiplication_count: 2305
clear_multiplication_count_per_parameter: {
    LweSecretKeyParam(dimension=1430): 2305
}
encrypted_negation_count: 3
encrypted_negation_count_per_parameter: {
    LweSecretKeyParam(dimension=1430): 3
}
size_of_secret_keys: 11440
size_of_bootstrap_keys: 0
size_of_keyswitch_keys: 0
p_error: 4.89428633255962e-13
global_p_error: 1.4682858997671677e-12
complexity: 1430.0
--------------------------------------------------------------------------------

Run time: 116.7409 seconds
Macro F1: 0.4381
F1 score for negative class